In [1]:
import pdblp

con = pdblp.BCon(debug=True, port=8194, timeout=5000)
con.start()

pdblp.pdblp:INFO:Event Type: 'SESSION_STATUS'
pdblp.pdblp:INFO:Message Received:
SessionConnectionUp = {
    server = "localhost:8194"
    serverId = "bbcomm-psd-finmath-15-606707"
    encryptionStatus = "Clear"
    compressionStatus = "Uncompressed"
}

pdblp.pdblp:INFO:Event Type: 'SESSION_STATUS'
pdblp.pdblp:INFO:Message Received:
SessionStarted = {
    initialEndpoints[] = {
        initialEndpoints = {
            address = "localhost:8194"
        }
    }
}

pdblp.pdblp:INFO:Event Type: 'SERVICE_STATUS'
pdblp.pdblp:INFO:Message Received:
CID: {[ valueType=AUTOGEN classId=0 value=1 ]}
RequestId: f61cad2b-6470-c1f1-0000-e326aac00600
ServiceOpened = {
    serviceName = "//blp/refdata"
}

pdblp.pdblp:INFO:Event Type: 'SERVICE_STATUS'
pdblp.pdblp:INFO:Message Received:
CID: {[ valueType=AUTOGEN classId=0 value=3 ]}
RequestId: ca1ca72b-6471-c1f1-0000-e326aac00600
ServiceOpened = {
    serviceName = "//blp/exrsvc"
}



In [59]:
import pandas as pd

con.debug = False

start_date = '20221101'
end_date = '20241112'

russell_tickers = pd.read_csv(r'tickers_russell_1000.csv', header=0)
russell_tickers['Ticker'] = russell_tickers['Ticker'].str.split(' ').str[0]
russell_tickers['Ticker'] = russell_tickers['Ticker'].astype(str) + ' US Equity'
russell_tickers

,Ticker
0,VRSK US Equity
1,VZ US Equity
2,RCL US Equity
3,JPM US Equity
4,LII US Equity
...,...
1005,AWI US Equity
1006,LOPE US Equity
1007,AVT US Equity
1008,SAIC US Equity


In [61]:
asia_tickers = pd.read_csv(r'asia_tickers.csv', header=0)
asia_tickers['Ticker'] = asia_tickers['Ticker'] + ' ' + asia_tickers['Exchange'] + ' Equity'
asia_tickers

,Ticker,Exchange,Sector,Industry
0,360 AU Equity,AU,Information Technology,Software
1,A2M AU Equity,AU,Consumer Staples,Food Products
2,ABC AU Equity,AU,Materials,Construction Materials
3,ADH AU Equity,AU,Consumer Discretionary,Specialty Retail
4,AGL AU Equity,AU,Utilities,Multi-Utilities
...,...,...,...,...
995,9983 JP Equity,JP,Consumer Discretionary,Specialty Retail
996,9984 JP Equity,JP,Communication Services,Wireless Telecommunication Services
997,9987 JP Equity,JP,Health Care,Health Care Providers & Services
998,9989 JP Equity,JP,Consumer Staples,Consumer Staples Distribution & Retail


In [70]:
import warnings
warnings.filterwarnings("ignore")

def get_bbg_price_index(tickers):
    price_df = pd.DataFrame()
    
    for ticker in tickers:
        try:
            ret = con.bdh(ticker, ['TOT_RETURN_INDEX_NET_DVDS'], start_date, end_date)
            ret = ret[ret.columns[0]]
            rolling_mean = ret.rolling(window=30, min_periods=1).mean()
            filled_series = ret.fillna(rolling_mean)
            price_df[ticker] =  filled_series
        except:
            print(ticker)
    return price_df

In [72]:
russell_returns_df = get_bbg_price_index(list(russell_tickers['Ticker'])).pct_change()
russell_returns_df.dropna()

,VRSK US Equity,VZ US Equity,RCL US Equity,JPM US Equity,LII US Equity,KO US Equity,DIS US Equity,IBM US Equity,JNJ US Equity,LPLA US Equity,...,TWKS US Equity,ST US Equity,WDAY US Equity,IPGP US Equity,LIN US Equity,AWI US Equity,LOPE US Equity,AVT US Equity,SAIC US Equity,TMUS US Equity
date,,,,,,,,,,,,,,,,,,,,,
2022-11-02,-0.066898,0.009098,-0.044633,-0.009208,-0.014058,-0.013917,-0.039434,-0.009913,-0.015368,-0.003080,...,-0.080125,-0.034780,-0.053364,0.028311,-0.012845,-0.019081,-0.004252,-0.014525,0.002305,-0.014841
2022-11-03,-0.023386,-0.014850,0.082763,0.001811,0.019487,-0.000510,-0.025241,-0.017248,0.001702,0.016145,...,-0.063348,-0.010989,-0.022925,-0.012173,0.011076,-0.011145,0.007449,-0.002498,0.006256,-0.012332
2022-11-04,0.006954,0.002423,0.017296,0.027358,-0.006654,0.008166,0.003325,0.018517,0.004452,0.014596,...,-0.009662,0.051938,-0.054129,0.011517,0.038071,-0.002337,-0.001380,0.035312,-0.001829,0.004454
2022-11-07,0.011830,-0.001343,-0.049177,0.005280,0.013994,0.003881,0.008536,0.010076,0.008747,0.003447,...,-0.013415,0.014738,0.029330,-0.008767,0.014761,0.026453,0.015499,0.018142,0.026381,0.006383
2022-11-08,0.012523,0.018284,0.044222,0.000609,0.032946,0.001849,-0.005277,0.012289,0.004972,0.004928,...,0.029666,0.005326,0.026809,-0.027223,0.003892,0.019598,0.021775,0.020908,-0.003570,0.000801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06,-0.001966,-0.002424,0.075295,0.115445,-0.033310,-0.025547,0.023388,0.029050,-0.002968,0.117291,...,-0.013393,0.059529,0.043731,0.048634,0.017015,0.037625,0.145975,0.047453,0.001746,0.022368
2024-11-07,0.004335,-0.014333,-0.027003,-0.043228,0.028870,-0.000628,0.000404,0.000422,-0.007284,-0.010579,...,0.002262,0.000598,0.024543,0.001752,0.006139,-0.005091,0.021168,-0.000699,-0.001475,-0.003015
2024-11-08,0.016551,-0.002220,0.021540,0.002538,0.013998,0.004084,0.000910,0.000140,-0.008039,0.024106,...,0.013544,-0.011649,-0.005582,-0.022041,-0.012869,0.007736,0.054910,-0.009977,0.020344,0.016502


In [73]:
russell_returns_df.dropna().to_csv(r'russell_1000_retuns.csv', header=True, index=True)

In [74]:
asia_returns_df = get_bbg_price_index(list(asia_tickers['Ticker'])).pct_change()
asia_returns_df.dropna()

,360 AU Equity,A2M AU Equity,ABC AU Equity,ADH AU Equity,AGL AU Equity,AKE AU Equity,ALL AU Equity,ALQ AU Equity,ALU AU Equity,AMC AU Equity,...,9873 JP Equity,9882 JP Equity,9948 JP Equity,9956 JP Equity,9962 JP Equity,9983 JP Equity,9984 JP Equity,9987 JP Equity,9989 JP Equity,167A JP Equity
date,,,,,,,,,,,,,,,,,,,,,
2022-11-02,-0.002954,0.013462,0.026946,0.036866,0.015449,-0.039229,-0.010074,-0.008540,-0.002776,-0.042952,...,-0.000362,-0.010609,0.010137,0.004079,-0.015773,-0.001203,0.019927,-0.022590,0.011645,0.023433
2022-11-03,0.002963,0.041746,-0.049563,-0.031111,-0.020747,-0.024913,0.000803,-0.021533,-0.007795,-0.009206,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-11-04,0.014771,-0.012750,0.006135,0.032110,0.029661,0.060326,-0.005619,-0.005282,0.006734,-0.001161,...,0.011236,-0.020880,-0.008028,-0.012188,-0.027244,-0.004221,-0.022073,-0.016949,-0.020144,0.014208
2022-11-07,-0.016012,0.016605,0.009146,-0.017778,0.017833,0.001339,-0.004306,0.004425,-0.044872,0.001744,...,-0.002151,-0.001729,0.000000,0.017039,0.037891,0.015015,0.006710,0.007837,-0.004405,-0.000379
2022-11-08,-0.041420,0.039927,0.006042,0.013575,0.018868,0.033422,-0.000811,-0.000881,0.000584,0.002902,...,-0.005029,0.007506,0.009611,0.008088,0.009524,-0.004058,0.049689,0.012442,0.007375,0.015056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06,0.030895,-0.029197,0.000000,0.051169,-0.004713,0.000000,0.011532,0.026518,0.000000,0.005785,...,0.000000,-0.005115,-0.005619,0.000452,0.012659,0.025457,0.051313,-0.023201,-0.017013,0.006353
2024-11-07,0.003967,-0.024436,0.000000,-0.026201,0.000000,0.000000,0.000316,0.006120,0.000000,-0.008311,...,0.000000,0.001974,0.014088,0.018678,-0.032946,-0.032969,-0.019465,0.018757,0.000806,0.005880
2024-11-08,0.044337,0.017341,0.000000,-0.003859,0.005684,0.000000,0.019630,0.000672,0.000000,-0.002581,...,0.000000,-0.007919,-0.006363,-0.013141,0.004205,0.011502,0.015836,0.003879,-0.002391,-0.005451


In [75]:
asia_returns_df.dropna().to_csv(r'asia_retuns.csv', header=True, index=True)